In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [2]:
data = {
    "3-15-20": [0, 1],
    "3-16-20": [1, 2],
    "3-17-20": [2, 3],
    "3-18-20": [3, 4],
    "3-19-20": [4, 5],
    "3-20-20": [5, 6],
    "3-21-20": [6, 7],
    "3-22-20": [8, 9],
    "3-23-20": [10, 11],
    "3-24-20": [12, 13],
    "3-25-20": [14, 15],
    "3-26-20": [20, 21],
    "3-27-20": [44, 24],
    "3-28-20": [4, 9],
    "3-29-20": [45, 30],
    "3-30-20": [8, 10],
    "3-31-20": [6, 9],
    "4-1-20": [77, 55],
    "4-2-20": [33, 47],
    "4-3-20": [9, 10],
}

df = pd.DataFrame(data=data, index=["4100", "4200"])
df.head()

,3-15-20,3-16-20,3-17-20,3-18-20,3-19-20,3-20-20,3-21-20,3-22-20,3-23-20,3-24-20,3-25-20,3-26-20,3-27-20,3-28-20,3-29-20,3-30-20,3-31-20,4-1-20,4-2-20,4-3-20
4100,0,1,2,3,4,5,6,8,10,12,14,20,44,4,45,8,6,77,33,9
4200,1,2,3,4,5,6,7,9,11,13,15,21,24,9,30,10,9,55,47,10


In [3]:
WINDOW_SIZE = 14

augmented_data = {x:[] for x in range(WINDOW_SIZE)}
augmented_df = pd.DataFrame(data=augmented_data)
for index, row in df.iterrows():
    for i, col in enumerate(df.columns[:-WINDOW_SIZE]):
        series = row[df.columns[i:i+WINDOW_SIZE]].reset_index(name=f"{index}_{df.columns[i+WINDOW_SIZE-1]}", drop=True)
        augmented_df.at[f"{index}_{df.columns[i+WINDOW_SIZE-1]}", :] = series.to_dict()
augmented_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
4100_3-28-20,0.0,1.0,2.0,3.0,4.0,5.0,6.0,8.0,10.0,12.0,14.0,20.0,44.0,4.0
4100_3-29-20,1.0,2.0,3.0,4.0,5.0,6.0,8.0,10.0,12.0,14.0,20.0,44.0,4.0,45.0
4100_3-30-20,2.0,3.0,4.0,5.0,6.0,8.0,10.0,12.0,14.0,20.0,44.0,4.0,45.0,8.0
4100_3-31-20,3.0,4.0,5.0,6.0,8.0,10.0,12.0,14.0,20.0,44.0,4.0,45.0,8.0,6.0
4100_4-1-20,4.0,5.0,6.0,8.0,10.0,12.0,14.0,20.0,44.0,4.0,45.0,8.0,6.0,77.0


In [4]:
X = augmented_df[range(0, 13)]
y = augmented_df[13]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [5]:
baseline = LinearRegression()
baseline.fit(X_train, y_train)

LinearRegression()